In [1]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.python.platform
import matplotlib.image as mpimg
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [9]:
NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 80
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 10
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [5]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract pixel patches into numpy arrays. Dim: (nb patch, 16, 16, 3)
training_data = extract_data(train_data_filename, 100)
training_labels = extract_labels(train_labels_filename, 100)

100%|██████████| 100/100 [00:01<00:00, 51.40it/s]


Loaded 100 training images


100%|██████████| 100/100 [00:00<00:00, 415.11it/s]


Loaded 100 groudtruth images


In [6]:
#Create training sets and validation sets
X_train, X_validation, y_train, y_validation = train_test_split(training_data, training_labels,\
                                                    train_size= TRAINING_SIZE/100, random_state=SEED)
print(X_train.shape)
print(X_validation.shape)

(50000, 16, 16, 3)
(12500, 16, 16, 3)


In [7]:
# Balance the categories such that there is the same number of background pixel
# as road pixel

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(X_train.shape)
X_train = X_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 37053 c1 = 12947
Balancing training data...
25894
(50000, 16, 16, 3)
Number of data points per class: c0 = 12947 c1 = 12947


In [8]:
X_train.shape

(25894, 16, 16, 3)

In [92]:
if RESTORE_MODEL:
    # Reconstruct the model identically.
    model = models.load_model("handmade_cnn_save/")

In [194]:
if not RESTORE_MODEL:
    # Create the model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(16, 16, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(8, 8, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))
model.summary()

In [196]:
if not RESTORE_MODEL: 
    # Train the model
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=NUM_EPOCHS ,validation_data=(X_validation, y_validation))

Train on 50000 samples, validate on 12500 samples
Epoch 1/10
50000/50000 [==============================] - 29s 576us/sample - loss: 0.5158 - accuracy: 0.7441 - val_loss: 0.5002 - val_accuracy: 0.7546
Epoch 2/10
50000/50000 [==============================] - 26s 517us/sample - loss: 0.4725 - accuracy: 0.7636 - val_loss: 0.4545 - val_accuracy: 0.7714
Epoch 3/10
50000/50000 [==============================] - 28s 564us/sample - loss: 0.4361 - accuracy: 0.7853 - val_loss: 0.4591 - val_accuracy: 0.7666
Epoch 4/10
50000/50000 [==============================] - 29s 574us/sample - loss: 0.3971 - accuracy: 0.8105 - val_loss: 0.4176 - val_accuracy: 0.8034
Epoch 5/10
50000/50000 [==============================] - 25s 500us/sample - loss: 0.3750 - accuracy: 0.8209 - val_loss: 0.3874 - val_accuracy: 0.8149
Epoch 6/10
50000/50000 [==============================] - 27s 536us/sample - loss: 0.3567 - accuracy: 0.8309 - val_loss: 0.3796 - val_accuracy: 0.8238
Epoch 7/10
50000/50000 [====================

In [197]:
if not RESTORE_MODEL:
    model.save("handmade_cnn_save")

INFO:tensorflow:Assets written to: handmade_cnn_save/assets


In [198]:
pred_train = model.predict(X_train)
print("Training error rate: {:.2f}%".format(error_rate(pred_train, y_train)))

Training error rate: 11.23%


In [199]:
pred_validation = model.predict(X_validation)
print("Validation error rate: {:.2f}%".format(error_rate(pred_validation, y_validation)))

Validation error rate: 18.40%


In [200]:
print("F1 score: {:.2f}".format(F1_score(y_train, pred_train)))

F1 score: 0.79


In [146]:
#Create concatenation and overlay of training images and predictions

print("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE + 1):
    pimg = get_prediction_with_groundtruth(model, train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(model, train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


In [179]:
test_dir = 'data/test_set_images/'
prediction_test_dir = "predictions_test/"

#Create the masks predicted for the test images
predict_test_masks(model, test_dir, prediction_test_dir)

Running prediction on test set


In [188]:
#Create the submission csv
masks_to_submission("submission.csv", prediction_test_dir)

100%|██████████| 50/50 [00:02<00:00, 20.46it/s]
